In [9]:
import os
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

In [10]:
from openai import OpenAI, completions
from settings import OpenAISettings

In [11]:
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())
console = Console()

In [18]:
def get_client() -> OpenAI:
    settings = OpenAISettings()
    client = OpenAI(
        api_key=settings.openai_api_key.get_secret_value(),
        base_url=str(settings.openai_base_url),
    )
    return client

In [25]:
def responses_variant():
    client = get_client()

    completion = client.responses.create(
        model="qwen/qwen2.5-vl-32b-instruct:free",
        instructions=(
        "Ты - молодой поэт, который только начинает писать стихи, но у тебя пока не очень получается."
        "Ты говоришь всем, какой ты крутой поэт и общаешься ТОЛЬКО стихами."
        "Пиши стихи в классическом формате четверостиший, тщательно следи за рифмой.")
        
    )

    print(completion.output_text)

In [26]:
messages = [
    {
        "role": "system",
        "content":(
            "Ты - молодой поэт, который только начинает писать стихи, но у тебя пока не очень получается."
            "Ты говоришь всем, какой ты крутой поэт и общаешься ТОЛЬКО стихами."
            "Пиши стихи в классическом формате четверостиший, тщательно следи за рифмой.")
    },
]

In [27]:
def completions_variant():
    client = OpenAI()
    completion = client.chat.completions.create(
        model="qwen/qwen2.5-vl-32b-instruct:free",
        messages=messages,
    )
    assistant_message = completion.choices[0].message.content
    messages.append(
        {
            "role": "assistant",
            "content": assistant_message,
        }
    )
    return assistant_message

In [28]:
def llm_chat():
    console.print(Panel(
        "[bold red]что вы хотите сделать?[/bold red]\n\n"
        "что можно сделать:  "
        "  [yellow]/exit[/yellow] - выход;  "
        "  [yellow]/clear[/yellow] - очистить историю;  "
        "  [yellow]/system текст[/yellow] - изменить системный промпт.",
        title="Поэтический вечер"
    ))
    
    while True:
        user_input = console.input("\n[bold white]ответ твари дрожащей:[/bold white] ").strip()
        print(user_input)
        
        if user_input.strip() == "":
            continue
            
        if user_input == "/exit":
            console.print("[yellow]не возвращайтесь...[/yellow]")
            break
            
        elif user_input == "/clear":
            system_message = messages[0]
            messages.clear()
            messages.append(system_message)
            console.print("[green]ушёл и пох[/green]")
            continue
            
        elif user_input.startswith("/system"):
            system_update = user_input[8:].strip()
            if messages and messages[0]["role"] == "system":
                messages[0]["content"] = system_update
            else:
                messages.insert(0, {"role": "system", "content": system_update})
                
            console.print(f"[green]новая роль:[/green] {user_input[8:]}")
            continue
        
        messages.append({
            "role": "user",
            "content": user_input,
        })
        
        console.print("\n[bold cyan]Генерирую рифмы...[/bold cyan]")
        response = completions_variant()
        
        markeddown_response = Markdown(response)
        console.print(Panel(
            markeddown_response,
            title="[bold red]Ответ отца поэзии:[/bold red]",
            border_style="cyan"
        ))

if __name__ == "__main__":
    llm_chat()

╭─────────────────────────────────────────────── Поэтический вечер ───────────────────────────────────────────────╮
│ что вы хотите сделать?                                                                                          │
│                                                                                                                 │
│ что можно сделать:    /exit - выход;    /clear - очистить историю;    /system текст - изменить системный        │
│ промпт.                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ответ твари дрожащей:

напиши стихотворение про даринку

Генерирую рифмы...

╭────────────────────────────────────────────── Ответ отца поэзии: ───────────────────────────────────────────────╮
│ Дама прекрасная, Дарина, Всюду, где она ходит. Словно молва облетает О ней сладкое воспоминанье.                │
│                                                                                                                 │
│ Золотые её волосы Светом солнца просияли, Взор её ясный и приветливый И сердца не променяли.                    │
│                                                                                                                 │
│ Лицо, как весенний цветок, И губки, как розовые чайки. Судьбой награждена она Красою солнца зорного.            │
│                                                                                                                 │
│ И, что особо порадует, Самая добрая из всех. Как добрае сердечко сердце, Дать бы всем в этом пример!            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ответ твари дрожащей:

/exit

не возвращайтесь...